# WeRateDogs Twitter Data Wrangling and Visualization
@dog_rates is a popular twitter account known for uploaded images of cute dogs and rating them (mostly above 10). In this project, we will be performing analysis on the data pulled from their twitter page. There are three (3) groups of data:
<ul>
    <li> Archive data (old tweets from 2015, 2016 and 2017 which is provided)</li>
<li>Image prediction (a collection of dog breed prediction based of the images pulled from the twitter page)</li>
    <li> More detailed tweet (pulled using ids from the archive) </li>
</ul>
After wrangling, we will be answering some questions and making beautiful visualizations. Lets get started !

In [ ]:
#import necessary packages and twitter api key details
from keys import consumer_key, consumer_secret, access_token, access_secret
import matplotlib.pyplot as plt
import requests
import tweepy
import pandas as pd
import seaborn as sns
import time
import json
import re
%matplotlib inline

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

## Gather

Use the request library to download the image predictions data and save as tsv

In [ ]:
r = requests.get("https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv")
with open("image_predictions.tsv", "w") as file:
    file.write(r.text)

In [ ]:
# read the saved image predictions data
df_image_pred = pd.read_csv("image_predictions.tsv", sep="\t")

Setup Tweepy package using imported twitter API details

In [ ]:
# setup tweepy
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

Directly download the WeRateDogs twitter archive data (twitter-archive-enhanced.csv)

In [ ]:
df_archive = pd.read_csv('twitter-archive-enhanced.csv')

In [ ]:
# make a list of tweet ids
tweet_ids = df_archive.tweet_id.tolist()

Use Tweepy to fetch more details of WeRateDogs twitter archive data and save (tweet_json.txt)

In [ ]:
#fetch tweets using tweet ids from archive-enhanced
failed_tweet_pull_count = 0
for id in tweet_ids:
    try:
        start = time.time()
        tweet = api.get_status(id, tweet_mode='extended')
        with open('tweet_json.txt', 'a') as file:
            json.dump(tweet._json, file)
            file.write("\n")
        end = time.time()
        print(end-start, ' seconds')
    except:
        failed_tweet_pull_count += 1
        print('Could not fetch tweet with id:', id)
        
print('Could not fetch ', failed_tweet_pull_count, ' Tweets')

Convert tweet_json.txt to a dataframe

In [ ]:
# read and append json to list
tweet_json_list = list()
with open("tweet_json.txt") as file_in:
    for line in file_in:
        tweet_json_list.append(json.loads(line))

In [ ]:
df_tweet = pd.DataFrame(tweet_json_list)

In [ ]:
df_tweet.to_csv('tweet_json.csv', index=False)

## Assess

### Programmatically view the three datasets in order to identify issues 

In [ ]:
# check for duplicated columns within the three datasets
all_columns = pd.Series(list(df_image_pred) + list(df_tweet) + list(df_archive))
all_columns[all_columns.duplicated()]

In [ ]:
# return random 15 rows of data in archive dataframe
df_archive.sample(15)

In [ ]:
#Use pandas info method to assess data
df_archive.info()

In [ ]:
#use pandas describe method
df_archive.describe()

In [ ]:
# Look for potential outliers with >10 rating denominator
df_archive[df_archive.rating_denominator > 10]

In [ ]:
#check for duplicates
df_archive.duplicated().sum()

In [ ]:
#check for retweets
df_archive[-df_archive.retweeted_status_id.isnull()]

In [ ]:
#check size of archive data (rows and columns)
df_archive.shape

Apply the same pandas methods to tweet dataframe

In [ ]:
df_tweet.info()

In [ ]:
#count num of distinct values in coordinates column
df_tweet.coordinates.value_counts()

In [ ]:
#count num of distinct values in geo column
df_tweet.geo.value_counts()

In [ ]:
#count num of distinct values in is_quote_status column
df_tweet.is_quote_status.value_counts()

In [ ]:
df_tweet.describe()

In [ ]:
# confirm that all tweets language is english
df_tweet[df_tweet_clean.lang != 'en']

In [ ]:
#check for tweet dataframe size
df_tweet.shape

Do the same foe image predictions data

In [ ]:
df_image_pred

In [ ]:
df_image_pred.info()

In [ ]:
df_image_pred.describe()

In [ ]:
# check for duplicated rows
df_image_pred.duplicated().sum()

In [ ]:
#size of image predictions data
df_image_pred.shape

#### Quality
##### `Tweets` table
- created_at column is an object not a datetime
- entire columns filled with null value
- repeated id column (id and id str)
- missing records (2327 instead of 2356)
- non original tweets (retweets, quotes, reply)
- Unwanted columns (only need created_at, id, retweet_count, favorite_count, full_text)

##### `Archive tweets` table
- non original tweets (retweets, quotes, reply)
- Null values in multiple columns
- Wrong datatype for timestamp, floofer, pupper and puppo columns
- Wrong value in rating columns (9/11 event mistaken as rating)
- Non 10 rating denominator value
- Incorrect value for dog stages

##### `Image Prediction` table
- Underscore used instead of space to seperate words (p1, p2 and p3 columns)
- No uniform character case in p1, p2 and p3 values
- Erroneous datatypes (p1_dog, p2_dog, p3_dog) 


#### Tidiness
- Same column bearing different title in another table (timestamp/created_at, id/tweet_id, text/full_text)
- Tweet text duplicated in tweet and archive table
- dog stages in four columns instead of one column in archive table
- multiple variables in entities and extended entities column of tweet table

## Clean

Make a copy of the three datasets 

In [ ]:
# Make a copy of the data
df_archive_clean = df_archive.copy()
df_tweet_clean = df_tweet.copy()
df_image_pred_clean = df_image_pred.copy()

### Missing Data

#### `Tweets table`: Missing records (2327 instead of 2356)

##### Define
compare the tweet ids in the archive df to fetched tweets ids, extract missing tweets and manually test some to confirm its non existence

##### Code

Extract tweets ids from archive df, tweet df and comapare to find missing tweets

In [ ]:
fetched_tweet_ids = df_tweet_clean.id.tolist()
archive_tweet_ids = df_archive_clean.tweet_id.tolist()
missing = list(set(archive_tweet_ids).difference(fetched_tweet_ids))

In [ ]:
print(missing)

#### Test

Attempt the pull the missing tweets again using tweepy

In [ ]:
for i in missing:
    try:
        tweet = api.get_status(id=i, tweet_mode='extended')
        print('Found')
    except:
        print('Not Found')

In [ ]:
df_archive_clean.sample()

### Tidiness

#### `Archive` Same column bearing different title in another table (timestamp/created_at, id/tweet_id, text/full_text)

##### Define
Rename the timestamp column to created_at, tweet_id to id and text to full_text

#### Code

Use rename() to change the column names of aformentioned columns

In [ ]:
df_archive_clean.rename(columns = {'timestamp': 'created_at', 'tweet_id': 'id', 'text': 'full_text'}, inplace=True)

In [ ]:
df_image_pred_clean.rename(columns = {'tweet_id': 'id'}, inplace=True)

#### Test

In [ ]:
#print out the columns
df_archive_clean.columns

#### `Archive` Tweet text duplicated in tweet and archive table

#### Define
Delete full text column in archive table

#### Code

In [ ]:
#drop full_text column
df_archive_clean.drop('full_text', axis=1, inplace=True)

#### Test

In [ ]:
#print column names
df_archive_clean.columns

#### `Archive` dog stages in three columns instead of one column in archive table

#### Define
Use pandas melt function to merge doggo, floofer, pupper and puppo column into one

#### Code

In [ ]:
#apply melt method
df_archive_clean = pd.melt(df_archive_clean, id_vars=['id', 'in_reply_to_status_id', 'in_reply_to_user_id', 'created_at',
       'source', 'retweeted_status_id', 'retweeted_status_user_id',
       'retweeted_status_timestamp', 'expanded_urls', 'rating_numerator',
       'rating_denominator', 'name'], var_name='stage', value_name='stage_value')


In [ ]:
# drop stage col and duplicate rows based on id
df_archive_clean.drop('stage', axis=1, inplace=True)
df_archive_clean = df_archive_clean.drop_duplicates('id')

Rename column `name` to `dog_name` and `stage_value` to `dog_stage`

In [ ]:
df_archive_clean.rename(columns = {'name': 'dog_name', 'stage_value': 'dog_stage'}, inplace=True)

In [ ]:
#check size of data
df_archive_clean.shape

#### Test

In [ ]:
df_archive_clean.head()

#### `Tweet` multiple variables in entities and extended entities column of tweet table

#### Define
Drop `entities` and  `extended_entities` columns

#### Code

In [ ]:
#drop both columns
df_tweet_clean.drop(['entities', 'extended_entities'], axis=1, inplace=True)

#### Test

In [ ]:
#check remaining columns
df_tweet_clean.columns

In [ ]:
#pull up random row from tweet df
df_tweet_clean.sample()

### Quality

#### `Tweet` created_at column is an object not a datetime

#### Define
convert created_at column to a datetime object

In [ ]:
df_tweet_clean['created_at'] = pd.to_datetime(df_tweet_clean['created_at'])

#### Test

In [ ]:
df_tweet_clean.dtypes

#### `Tweet` entire columns filled with null value (geo, coordinates, contributors)

#### Define 
geo is a deprecated feature while coordinates and contributors access might have been turned off by we_rate_dogs. Drop them.

#### Code

In [ ]:
#drop 3 columns (geo, coordinates, contributors)
df_tweet_clean.drop(['geo', 'coordinates', 'contributors'], axis=1, inplace=True)

#### Test

In [ ]:
df_tweet_clean.columns

In [ ]:
df_tweet_clean.shape

#### `Tweet` non original tweets (retweets, quotes, reply)

#### Define 
drop all tweets that are retweets, quotes and replies by checking rows with non null values in_reply_to_status_id and quoted_status_id columns. Also check for true value in retweeted column

#### Code

Drop all rows with tweet ids in `quoted_status_id` and `in_reply_to_status_id` columns

In [ ]:
df_tweet_clean.drop(df_tweet_clean.loc[-df_tweet_clean['in_reply_to_status_id'].isnull()].index, inplace=True)
df_tweet_clean.drop(df_tweet_clean.loc[-df_tweet_clean['quoted_status_id'].isnull()].index, inplace=True)

In [ ]:
df_tweet_clean[df_tweet_clean.retweeted == True]

#### Test

In [ ]:
df_tweet_clean.shape

In [ ]:
df_tweet_clean.sample()

In [ ]:
df_tweet_clean.columns

#### `Tweet` Remove Unwanted columns 

#### Define
Select created_at, id, retweet_count, favorite_count and full_text columns. Drop the rest.

In [ ]:
#drop all columns inside the column_names list
column_names = ['id_str', 'truncated',
       'display_text_range', 'source', 'in_reply_to_status_id',
       'in_reply_to_status_id_str', 'in_reply_to_user_id',
       'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'user', 'place',
       'is_quote_status', 'favorited',
       'retweeted', 'possibly_sensitive', 'possibly_sensitive_appealable',
       'lang', 'retweeted_status', 'quoted_status_id', 'quoted_status_id_str',
       'quoted_status_permalink', 'quoted_status']
df_tweet_clean.drop(column_names, axis=1, inplace=True)

#### Test

In [ ]:
df_tweet_clean.columns

#### `Archive` non original tweets (retweets, quotes, replies)

#### Define 
drop all tweets that are replies and retweets by checking rows with non null values in_reply_to_status_id and retweeted_status_user_id columns. 

#### Code

In [ ]:
#remove reply and retweets in archive table
df_archive_clean.drop(df_archive_clean.loc[-df_archive_clean['in_reply_to_status_id'].isnull()].index, inplace=True)
df_archive_clean.drop(df_archive_clean.loc[-df_archive_clean['retweeted_status_user_id'].isnull()].index, inplace=True)

#### Test

In [ ]:
df_archive_clean.in_reply_to_status_id.isnull().sum()

In [ ]:
df_archive_clean.retweeted_status_user_id.isnull().sum()

In [ ]:
df_archive_clean.shape

#### `Archive` null values in multiple columns

#### Define
Drop all columns with zero non null value

Remove columns filled with null values

In [ ]:
df_archive_clean.drop(['in_reply_to_status_id', 'in_reply_to_user_id', 'retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp'], axis=1, inplace=True)

#### Test

In [ ]:
df_archive_clean.info()

#### `Archive` Wrong datatype for timestamp, floofer, pupper and puppo columns

#### Define 
convert timestamp(now created_at) to type datetime. floofer, pupper and puppo columns have been merged.

#### Code

Change `created_at` column to type datetime

In [ ]:
df_archive_clean['created_at'] = pd.to_datetime(df_archive_clean['created_at'])

#### Test

In [ ]:
df_archive_clean.dtypes

#### `Archive` Wrong value in rating columns (9/11 event mistaken as rating)

#### Define
merge archive and tweet df. replace rows that has rating_numerator and 11 as rating_denominator with the correct rating found in full_text column using regex to extract the values

#### Code 

Left join `archive df` to `tweet df` on `id` and `created_at` column

In [ ]:
df_archive_tweet_merge = pd.merge(df_tweet_clean, df_archive_clean, on=['id', 'created_at'], how='left')

In [ ]:
df_archive_tweet_merge[df_archive_tweet_merge.rating_numerator.isnull()]

#### Notice
Upon examining the data below, I found out that all data with NaN values in rating_numerator, rating_denominator and name at the same time are all retweets. Will be removing them

In [ ]:
#drop tweets that are retweets
df_archive_tweet_merge.drop(df_archive_tweet_merge.loc[df_archive_tweet_merge['rating_numerator'].isnull()].index, inplace=True)


Use regex to extract rating from full_text and store in `new_rating` column. Seperate the two values using python split function and assign values to `rating_numerator` and `rating_denominator` columns. Drop `new_rating` column

In [ ]:
df_archive_tweet_merge['new_rating'] = df_archive_tweet_merge.full_text.str.extract('([\d]*/10)', expand=True)
df_archive_tweet_merge['rating_numerator'], df_archive_tweet_merge['rating_denominator'] = df_archive_tweet_merge['new_rating'].str.split('/', 1).str
df_archive_tweet_merge.drop('new_rating', axis=1, inplace=True)

#### Test

In [ ]:
df_archive_tweet_merge.sample(4)

#### Non 10 rating denominator value

#### Define
There are 12 rows with non rating denominators with have been replaced with NaN. I will drop them.

#### Code

Drop 12 samples of data with null values in the rating_denominator column

In [ ]:
df_archive_tweet_merge.drop(df_archive_tweet_merge.loc[df_archive_tweet_merge['rating_denominator'].isnull()].index, inplace=True)

#### Test

In [ ]:
df_archive_tweet_merge.shape

In [ ]:
df_archive_tweet_merge.info()

#### Incorrect value for dog stage

#### Define
Use regex to filter out puppo, pupper, floofer and doggo where mentioned in the full_text column. Replace the NaN values with Unknown

In [ ]:
#apply regex
df_archive_tweet_merge['dog_stage'] = df_archive_tweet_merge.full_text.str.extract(r'(puppo|pupper|floofer|doggo)', flags=re.IGNORECASE, expand=True)

In [ ]:
#change all text in `dog_stage` column to lowercase and replace null values with unknown
df_archive_tweet_merge['dog_stage'] = df_archive_tweet_merge['dog_stage'].str.lower()
df_archive_tweet_merge['dog_stage'].fillna('Unknown', inplace=True)

#### Test

In [ ]:
df_archive_tweet_merge['dog_stage'].value_counts()

In [ ]:
df_archive_tweet_merge.sample(2)

#### `Image Prediction` Underscore used instead of space to seperate words (p1, p2 and p3 columns)

#### Define
Use `Series.str.replace` to replace all underscores with space

#### Code

In [ ]:
#replace underscore (_) with space
df_image_pred_clean.p1 = df_image_pred_clean.p1.str.replace('_', ' ')
df_image_pred_clean.p2 = df_image_pred_clean.p2.str.replace('_', ' ')
df_image_pred_clean.p3 = df_image_pred_clean.p3.str.replace('_', ' ')

#### Test

In [ ]:
df_image_pred_clean.head()

#### `Image Prediction` No uniform character case in p1, p2 and p3 values

#### Define
Change p1, p2 and p3 to lower case using `str.lower`

#### Code

In [ ]:
#change all text in p1, p2 and p3 to lowercase
df_image_pred_clean.p1 = df_image_pred_clean.p1.str.lower()
df_image_pred_clean.p2 = df_image_pred_clean.p2.str.lower()
df_image_pred_clean.p3 = df_image_pred_clean.p3.str.lower()

#### Test

In [ ]:
df_image_pred_clean.sample(3)

#### `Image Prediction` Erroneous datatypes (p1_dog, p2_dog, p3_dog)

#### Define
Convert aformentioned columns to type category

In [ ]:
#convert p1_dog, p2_dog and p3_dog columns to type category
df_image_pred_clean.p1_dog = df_image_pred_clean.p1_dog.astype('category')
df_image_pred_clean.p2_dog = df_image_pred_clean.p2_dog.astype('category')
df_image_pred_clean.p3_dog = df_image_pred_clean.p3_dog.astype('category')

#### Test

In [ ]:
df_image_pred_clean.info()

### Store Data

Merge the three datasets (recall that tweet and archive has been merged) and save as `twitter_archive_master.csv`

In [ ]:
#merge three datasets
twitter_archive_master_merge = pd.merge(df_image_pred_clean, df_archive_tweet_merge, on='id')

In [ ]:
#check for null values
twitter_archive_master_merge.isnull().values.any()

In [ ]:
# save data as csv
twitter_archive_master_merge.to_csv('twitter_archive_master.csv', index=False)

In [ ]:
#save cleaned image predictions data as csv
df_image_pred_clean.to_csv('image_pred_updated.csv', index=False)

In [ ]:
#save combination of tweet and archive data as csv
df_archive_tweet_merge.to_csv('archive_tweet_merge_updated.csv', index=False)

## Visualization and insights
> Having cleaned the data, lets now discover some insights by posing couple of questions and answering them both programmatically and graphically

In [ ]:
#read the master dataset and store it in variable df
df = pd.read_csv('twitter_archive_master.csv')

In [ ]:
#change created_at column to type datetime
df['created_at'] = pd.to_datetime(df['created_at'])

### Using the dataset above, I intend to answer these questions:
- Which dog breed is most loved (based on likes) ?
- Least rated dog in year 2016 ?
- Most retweeted dog breed ?
- Most prominent word used to describe dogs (word cloud)

In [ ]:
#create a new column titled year and store the individual year each tweet was made
df['year'] = df['created_at'].dt.year

In [ ]:
#view distinct years present 
df.year.value_counts()

### Insight 1: Which dog breed is most loved (based on likes) ?
> I will find the sum amount of likes for each dog breed using the p1 and p1_dog column and then the max of all the averages. p1_dog column must be True for the breed predicted to be evaluated. 

In [ ]:
#select all rows with p1 column == true
p1_dog_true = df[df.p1_dog == True]

In [ ]:
#view the first five row
p1_dog_true.head()

Use pandas groupby to perform summation of favorite_count for each dog breed in p1 columns. Plot the data using horizonatal bar chart

In [ ]:
dog_like_rating = p1_dog_true.groupby(['p1'])['favorite_count'].sum().sort_values(ascending=False)
plt.title('Top 20 Dogs breeds based on likes', size = 15)
rating_fig = dog_like_rating[:20].plot(kind="barh",color="green")
rating_fig.figure.set_size_inches(12, 9)
rating_fig.set_ylabel('Dog breeds', color = 'black', fontsize = '13')
rating_fig.set_xlabel('Num of Likes', color = 'black', fontsize = '13')
pd.DataFrame(dog_like_rating[:20])

> From the analysis above, Golden retriever is the most loved dog breed on weRateDogs twitter page with 1459893 total amount of likes

### Insight 2: Least rated dog in year 2016 ?
> Check for abnormal ratings and remove them before getting the average rating of all p1 dog breeds. 

In [ ]:
# count number of disting values in rating_numerator
p1_dog_true.rating_numerator.value_counts()

In [ ]:
#select tweets made in 2016
p1_dog_true_2016 = p1_dog_true[p1_dog_true.year == 2016]

In [ ]:
#find the mean of rating numerators for each breed
least_rated_2016 = p1_dog_true_2016.groupby(['p1'])['rating_numerator'].mean().sort_values(ascending=True)

In [ ]:
#view result
least_rated_2016

> bloodhound is the least rated dog breed with average rating of 7.5 in the year 2016.

### Insight 3: Most retweeted dog breed ?

Sum up retweet count for each dog breed. Use pie chart to plot values

In [ ]:
#retweet value summation
dog_retweet_rating = p1_dog_true.groupby(['p1'])['retweet_count'].sum().sort_values(ascending=False)

In [ ]:
#plot the data
dog_retweet_rating.iloc[:10].plot(kind='pie', radius=3);
plt.title('Top 10 Dogs breeds and their amount of twitter retweets', size=15, pad=200)
pd.DataFrame(dog_retweet_rating[:10])

> Golden retriever is the most retweeted dog breed on weRateDogs page. 

### Word Cloud for prominent dog descriptions 

Use word cloud and matplotlib to view popular words used by WeRateDogs to describe dogs 

In [ ]:
from wordcloud import WordCloud,STOPWORDS
text=p1_dog_true.full_text.values
wordcloud = WordCloud(
    width = 3000,
    height = 2000,
    background_color = 'black',
    stopwords = STOPWORDS).generate(str(text))
fig = plt.figure(
    figsize = (40, 30),
    facecolor = 'k',
    edgecolor = 'k')
plt.imshow(wordcloud, interpolation = 'bilinear')
plt.axis('off')
plt.tight_layout(pad=0)
plt.show()

## Conclusion

The following insights were discovered after visualization
- Bloodhound is the least rated dog breed with average rating of 7.5 in the year 2016.
- Golden retriever is the most loved and retweeted dog breed on weRateDogs twitter page.